In [1]:
if sc:
    sc.stop()

In [2]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("22125094_lab6")
sc = SparkContext(conf = conf)

sc.getConf().getAll()

[('spark.app.submitTime', '1765458003968'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.driver.port', '33537'),
 ('spark.org.apache.hadoop

In [ ]:
retail_data = sc.textFile("hdfs://name:9000/Lab6_Datasets/online_retail.csv")

header = retail_data.first()
retail_data = retail_data.filter(lambda x : x != header)
retail_data = retail_data.map(lambda x : x.split(","))  

retail_data.collect()

In [ ]:
def parse_row(x):
    try:
        for i in x:
            if i == "":
                return None
        return (
            int(x[0]),
            x[1],
            x[2],
            x[3],
            int(x[4]),
            x[5],
            float(x[6]),
            x[7],
            x[8]
        )
    except:
        return None

retail_parsed = retail_data.map(parse_row).filter(lambda x: x is not None)
retail_parsed.count()

In [ ]:
rfm = retail_parsed.map(lambda x: (x[7], x[1], x[5], x[4] * x[6]))
rfm.take(5)

In [ ]:
from datetime import datetime

def parse_date(x):
    cus_id, inv_no, inv_date, total = x
    inv_dt = datetime.strptime(inv_date, "%m/%d/%Y %H:%M")
    return (cus_id, inv_no, inv_dt, total)

rfm = rfm.map(parse_date)
rfm.take(5)

Compute RFM

In [ ]:
last_buy = rfm.map(lambda x: (x[0], x[2])).reduceByKey(lambda a, b : max(a, b))
latest = last_buy.map(lambda x: x[1]).max()
last_buy.take(5)


In [ ]:
recency = last_buy.map(lambda x: (x[0], (latest - x[1]).days))
recency.take(5)


In [ ]:
frequency = rfm.map(lambda x: (x[0], x[1])).distinct()
frequency = frequency.map(lambda x: (x[0], 1)).reduceByKey(lambda a, b : a + b)
frequency.take(5)

In [ ]:
monetary = rfm.map(lambda x: (x[0], x[3])).reduceByKey(lambda a, b : a + b)
monetary.take(5)

In [ ]:
rfm_rdd = recency.join(frequency).join(monetary)
rfm_rdd.take(5)

In [ ]:
rfm_final = rfm_rdd.map(lambda x: (float(x[1][0][0]), float(x[1][0][1]), float(x[1][1]))).collect()
rfm_final

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

rfm_array = np.array(rfm_final)

scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_array)
rfm_scaled

array([[-0.75536054,  1.06406227, -0.1609506 ],
       [-0.89441086,  1.49277746,  0.30825616],
       [-0.09983759, -0.2220833 , -0.011134  ],
       ...,
       [ 0.58548186, -0.3292621 , -0.19704634],
       [ 0.62521053, -0.43644089, -0.19312035],
       [ 0.52588887, -0.43644089, -0.21215216]])

In [ ]:
# n, d = rfm_scaled.shape

ALPHA_THRESHOLD = 2.0
EUCLIDEAN_THRESHOLD = 2.0
MERGE_THRESHOLD = 2.0

# DS = []
# CS = []
# RS = []

In [132]:
class ClusterSummary:
    def __init__(self, d):
        self.N = 0
        self.SUM = np.zeros(d)
        self.SUMSQ = np.zeros(d)

    def add_point(self, point):
        self.N += 1
        self.SUM += point
        self.SUMSQ += point ** 2
        
    def add_points(self, points):
        points = np.array(points)
        self.N += len(points)
        self.SUM += np.sum(points, axis=0)
        self.SUMSQ += np.sum(points ** 2, axis=0)
        
    def merge(self, other_summary):
        self.N += other_summary.N
        self.SUM += other_summary.SUM
        self.SUMSQ += other_summary.SUMSQ
        
    def get_centroid(self):
        if self.N == 0: 
            return np.zeros(d)
        return self.SUM / self.N
        
    def get_variance(self):
        if self.N == 0: 
            return np.zeros(d)
        return (self.SUMSQ / self.N) - (self.SUM / self.N) ** 2
        
    def get_std(self):
        return np.sqrt(np.abs(self.get_variance()))
    
def mahalanobis_dist(point, centroid, std):
    if np.any(std < 1e-6):
        # return np.linalg.norm(point - centroid)
        std = std + 1e-6
    return np.sqrt(np.sum(((point - centroid) / std) ** 2))


In [ ]:
def init_sets(first_chunk, k):
    n, d = first_chunk.shape
    num_clusters = k
    initial_centroids, labels = KMeans(first_chunk, num_clusters)
    DS = []
    CS = []
    RS = []
    for i in range(num_clusters):
        cluster_points = first_chunk[labels == i]
        if len(cluster_points) >= 1: 
            cs = ClusterSummary(d)
            cs.add_points(cluster_points) 
            DS.append(cs)
    return DS, CS, RS

In [134]:
def can_merge(c1, c2, d):
    std1 = c1.get_std()
    std2 = c2.get_std()
    merged = ClusterSummary(d)
    merged.merge(c1)
    merged.merge(c2)

    std_merged = merged.get_std()
    centroid1 = c1.get_centroid()
    centroid2 = c2.get_centroid()
    threshold = MERGE_THRESHOLD * np.sqrt(d)
    dist = mahalanobis_dist(centroid1, centroid2, std_merged)
    return dist < threshold
    # return np.all(std_merged <= (std1 + std2) * MERGE_THRESHOLD)

def merge_CS_clusters(CS, d):
    merged = True
    while merged:
        merged = False
        i = 0
        while i < len(CS):
            j = i + 1
            while j < len(CS):
                if can_merge(CS[i], CS[j], d):
                    CS[i].merge(CS[j])
                    CS.pop(j)
                    merged = True
                else:
                    j += 1
            i += 1

In [135]:
def process_remaining_chunks(DS, CS, RS, chunks, d):
    for i, chunk in enumerate(chunks[1:]):
        merge_CS_clusters(CS, d)
        print(f"Processing Chunk {i+2} ({len(chunk)} points)...")
        for point in chunk:
            # DS
            min_dist = float('inf')
            best_ds = -1
            for idx, cluster in enumerate(DS):
                dist = mahalanobis_dist(point, cluster.get_centroid(), cluster.get_std())
                if dist < min_dist:
                    min_dist = dist
                    best_ds = idx
            if min_dist < ALPHA_THRESHOLD * np.sqrt(d):
                DS[best_ds].add_point(point)
                continue
            # CS
            min_dist = float('inf')
            best_cs = -1
            for idx, cluster in enumerate(CS):
                dist = mahalanobis_dist(point, cluster.get_centroid(), cluster.get_std())
                if dist < min_dist:
                    min_dist = dist
                    best_cs = idx
            if min_dist < ALPHA_THRESHOLD * np.sqrt(d):
                CS[best_cs].add_point(point)
                continue
            
            formed = False
            for r_idx, r in enumerate(RS):
                dist = np.linalg.norm(point - r)
                if dist < EUCLIDEAN_THRESHOLD:
                    cs = ClusterSummary(d)
                    cs.add_point(point)
                    cs.add_point(r)
                    CS.append(cs)
                    RS.pop(r_idx)
                    formed = True
                    break

            if formed:
                continue
            # RS
            RS.append(point)

In [136]:
def final_merge(DS, CS, RS, d):
    print("Finalizing...")
    # CS into DS
    merge_CS_clusters(CS, d)
    for cs in CS:
        min_dist = float('inf')
        best_idx = -1
        for idx, ds in enumerate(DS):
            dist = mahalanobis_dist(cs.get_centroid(), ds.get_centroid(), ds.get_std())
            if dist < min_dist:
                min_dist = dist
                best_idx = idx
        # force merge to nearest DS
        if best_idx != -1:
            DS[best_idx].merge(cs)

    for point in RS:
        min_dist = float('inf')
        best_idx = -1
        for idx, ds in enumerate(DS):
            dist = mahalanobis_dist(point, ds.get_centroid(), ds.get_std())
            if dist < min_dist:
                min_dist = dist
                best_idx = idx
        if best_idx != -1:
            DS[best_idx].add_point(point)
    CS.clear()
    RS.clear()
    print("Clustering Complete.")
    for i, ds in enumerate(DS):
        print(f"Cluster {i}: N={ds.N} Centroid={ds.get_centroid()}")


In [137]:
total_points = len(rfm_scaled)
chunk_size = int(total_points / 10) # 10 chunks 
chunks = [rfm_scaled[i:i + chunk_size] for i in range(0, total_points, chunk_size)]

In [138]:
def compute_bfr_wcss(data, DS):
    wcss = 0.0
    for p in data:
        min_dist = float("inf")
        for ds in DS:
            centroid = ds.get_centroid()
            dist = np.linalg.norm(p - centroid) ** 2
            if dist < min_dist:
                min_dist = dist
        wcss += min_dist
    return wcss

In [139]:
def run_bfr(chunks, k):
    first_chunk = chunks[0]
    _, d = chunks[0].shape

    DS, CS, RS = init_sets(first_chunk, k)

    print("Initialization done.")
    print("DS:", len(DS), "CS:", len(CS), "RS:", len(RS))

    process_remaining_chunks(DS, CS, RS, chunks, d)

    print("Streaming processing done.")
    print("DS:", len(DS), "CS:", len(CS), "RS:", len(RS))

    final_merge(DS, CS, RS, d)

    print("Final merge done.")
    print("DS:", len(DS), "CS:", len(CS), "RS:", len(RS))
    return DS

In [ ]:

DS = run_bfr(chunks, k)